# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

~~[SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[ResNet34](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)~~
~~[PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)~~
~~[InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[BNInception](https://github.com/Cadene/pretrained-models.pytorch#bninception)~~
~~[InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)~~
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet131](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)


 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc
from tqdm import tqdm

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import *
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
2,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
3,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
4,49850884a00703afe5aab78c3ce074d2d4acae30.jpg,../data/cleaned/none/49850884a00703afe5aab78c3...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 46/46 [00:09<00:00,  5.01it/s]


Image reading complete.
Image array shape: (299, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--lr", type=float, default=LEARNING_RATE)
    parser.add_argument("--weight_decay", type=float, default=WEIGHT_DECAY)
    parser.add_argument("--epsilon", type=float, default=EPSILON)
    parser.add_argument("--amsgrad", type=bool, default=AMSGRAD)
    parser.add_argument("--betas", default=BETAS)
    parser.add_argument("--eta_min", type=float, default=ETA_MIN)
    parser.add_argument("--t_max", type=float, default=T_MAX)
    parser.add_argument("--t_mult", type=float, default=T_MULT)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = se_resnext101_32x4d()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.hparams.lr, 
            betas=self.hparams.betas, 
            eps=self.hparams.epsilon, 
            weight_decay=self.hparams.weight_decay,
            amsgrad=self.hparams.amsgrad
        )
        self.scheduler = WarmRestart(
            self.optimizer, 
            T_max=self.hparams.t_max, 
            T_mult=self.hparams.t_mult, 
            eta_min=self.hparams.eta_min
        )
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=self.hparams.t_mult)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())

        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()
torch.cuda.empty_cache()

log_notes = "revert hparams to 2022_12_04_17:37:29 fold 2, t_max and t_mult constants created"

# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Log parameters
logger.info(f"backbone: {hparams.backbone}")
logger.info(f"device_name: {hparams.device_name}")
logger.info(f"gpus: {hparams.gpus}")
logger.info(f"n_workers: {hparams.n_workers}")
logger.info(f"image_size: {hparams.image_size}")
logger.info(f"seed: {hparams.seed}")
logger.info(f"min_epochs: {hparams.min_epochs}")
logger.info(f"max_epochs: {hparams.max_epochs}")
logger.info(f"patience: {hparams.patience}")
logger.info(f"train_batch_size: {hparams.train_batch_size}")
logger.info(f"val_batch_size: {hparams.val_batch_size}")
logger.info(f"n_splits: {hparams.n_splits}")
logger.info(f"test_size: {hparams.test_size}")
logger.info(f"learning rate: {hparams.lr}")
logger.info(f"weight_decay: {hparams.weight_decay}")
logger.info(f"epsilon: {hparams.epsilon}")
logger.info(f"amsgrad: {hparams.amsgrad}")
logger.info(f"betas: {hparams.betas}")
logger.info(f"precision: {hparams.precision}")
logger.info(f"gradient_clip_val: {hparams.gradient_clip_val}")
logger.info(f"eta_min: {hparams.eta_min}")
logger.info(f"t_max: {hparams.t_max}")
logger.info(f"t_mult: {hparams.t_mult}")
logger.info(f"log_dir: {hparams.log_dir}")
logger.info(f"log_name: {hparams.log_name}")

# Log any notes if they exist
if "log_notes" in locals():
    logger.info(f"Notes: {log_notes}")


# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
val_loss_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_loss", 
        patience=hparams.patience, 
        mode="min", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
            
    # Save val scores
    val_loss_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
val_loss_scores = [i.item() for i in val_loss_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [val_loss_scores],
            "mean_score": [np.mean(val_loss_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {val_loss_scores}")
logger.info("Training complete.")

[2022-12-06 19:05:03] 1035762479.py[  11] : INFO  backbone: se_resnext101_32x4d
[2022-12-06 19:05:03] 1035762479.py[  12] : INFO  device_name: NVIDIA GeForce RTX 3090
[2022-12-06 19:05:03] 1035762479.py[  13] : INFO  gpus: [0]
[2022-12-06 19:05:03] 1035762479.py[  14] : INFO  n_workers: 128
[2022-12-06 19:05:03] 1035762479.py[  15] : INFO  image_size: [1024, 1024]
[2022-12-06 19:05:03] 1035762479.py[  16] : INFO  seed: 14
[2022-12-06 19:05:03] 1035762479.py[  17] : INFO  min_epochs: 75
[2022-12-06 19:05:03] 1035762479.py[  18] : INFO  max_epochs: 100
[2022-12-06 19:05:03] 1035762479.py[  19] : INFO  patience: 11
[2022-12-06 19:05:03] 1035762479.py[  20] : INFO  train_batch_size: 4
[2022-12-06 19:05:03] 1035762479.py[  21] : INFO  val_batch_size: 4
[2022-12-06 19:05:03] 1035762479.py[  22] : INFO  n_splits: 3
[2022-12-06 19:05:03] 1035762479.py[  23] : INFO  test_size: 0.1
[2022-12-06 19:05:03] 1035762479.py[  24] : INFO  learning rate: 0.0001
[2022-12-06 19:05:03] 1035762479.py[  25] :

[2022-12-06 19:34:24] 1068281035.py[  95] : INFO  0-37 | lr : 0.000054 | val_loss : 1.5904 | val_roc_auc : 0.7750 | data_load_times : 46.55 | batch_run_times : 47.11
[2022-12-06 19:35:09] 1068281035.py[  95] : INFO  0-38 | lr : 0.000051 | val_loss : 1.5450 | val_roc_auc : 0.7639 | data_load_times : 45.26 | batch_run_times : 45.83
[2022-12-06 19:35:55] 1068281035.py[  95] : INFO  0-39 | lr : 0.000048 | val_loss : 1.5039 | val_roc_auc : 0.7685 | data_load_times : 45.13 | batch_run_times : 45.67
[2022-12-06 19:36:42] 1068281035.py[  95] : INFO  0-40 | lr : 0.000044 | val_loss : 1.6516 | val_roc_auc : 0.7565 | data_load_times : 46.74 | batch_run_times : 47.34
[2022-12-06 19:37:28] 1068281035.py[  95] : INFO  0-41 | lr : 0.000041 | val_loss : 1.6958 | val_roc_auc : 0.7421 | data_load_times : 40.02 | batch_run_times : 40.63
[2022-12-06 19:38:14] 1068281035.py[  95] : INFO  0-42 | lr : 0.000038 | val_loss : 1.6975 | val_roc_auc : 0.7575 | data_load_times : 45.51 | batch_run_times : 46.06
[202

Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:00:27] 1068281035.py[  95] : INFO  0-71 | lr : 0.000098 | val_loss : 1.9354 | val_roc_auc : 0.7312 | data_load_times : 46.85 | batch_run_times : 47.46
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:01:12] 1068281035.py[  95] : INFO  0-72 | lr : 0.000098 | val_loss : 1.7491 | val_roc_auc : 0.7607 | data_load_times : 43.48 | batch_run_times : 44.06
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:01:58] 1068281035.py[  95] : INFO  0-73 | lr : 0.000098 | val_loss : 1.7306 | val_roc_auc : 0.7791 | data_load_times : 46.37 | batch_run_times : 47.06
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training

Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:34:57] 1068281035.py[  95] : INFO  1-39 | lr : 0.000048 | val_loss : 1.6256 | val_roc_auc : 0.7924 | data_load_times : 48.02 | batch_run_times : 48.59
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:35:45] 1068281035.py[  95] : INFO  1-40 | lr : 0.000044 | val_loss : 1.6205 | val_roc_auc : 0.7982 | data_load_times : 44.99 | batch_run_times : 45.57
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:36:33] 1068281035.py[  95] : INFO  1-41 | lr : 0.000041 | val_loss : 1.6518 | val_roc_auc : 0.7882 | data_load_times : 45.35 | batch_run_times : 45.91
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training

Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:57:14] 1068281035.py[  95] : INFO  1-67 | lr : 0.000099 | val_loss : 1.9830 | val_roc_auc : 0.7065 | data_load_times : 44.24 | batch_run_times : 44.83
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:58:01] 1068281035.py[  95] : INFO  1-68 | lr : 0.000099 | val_loss : 1.8192 | val_roc_auc : 0.7577 | data_load_times : 46.62 | batch_run_times : 47.33
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 20:58:48] 1068281035.py[  95] : INFO  1-69 | lr : 0.000099 | val_loss : 1.8258 | val_roc_auc : 0.7684 | data_load_times : 44.93 | batch_run_times : 45.56
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training

[2022-12-06 21:31:41] 1068281035.py[  95] : INFO  2-34 | lr : 0.000064 | val_loss : 1.4396 | val_roc_auc : 0.8318 | data_load_times : 47.79 | batch_run_times : 48.34
[2022-12-06 21:32:31] 1068281035.py[  95] : INFO  2-35 | lr : 0.000061 | val_loss : 1.4270 | val_roc_auc : 0.8299 | data_load_times : 46.87 | batch_run_times : 47.42
[2022-12-06 21:33:21] 1068281035.py[  95] : INFO  2-36 | lr : 0.000057 | val_loss : 1.4579 | val_roc_auc : 0.8154 | data_load_times : 45.92 | batch_run_times : 46.52
[2022-12-06 21:34:10] 1068281035.py[  95] : INFO  2-37 | lr : 0.000054 | val_loss : 1.4520 | val_roc_auc : 0.8054 | data_load_times : 45.52 | batch_run_times : 46.10
[2022-12-06 21:34:59] 1068281035.py[  95] : INFO  2-38 | lr : 0.000051 | val_loss : 1.3558 | val_roc_auc : 0.8334 | data_load_times : 44.66 | batch_run_times : 45.24
[2022-12-06 21:35:48] 1068281035.py[  95] : INFO  2-39 | lr : 0.000048 | val_loss : 1.4170 | val_roc_auc : 0.8085 | data_load_times : 44.67 | batch_run_times : 45.25
[202

Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 22:01:03] 1068281035.py[  95] : INFO  2-70 | lr : 0.000099 | val_loss : 1.6231 | val_roc_auc : 0.7943 | data_load_times : 45.63 | batch_run_times : 46.20
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 22:01:52] 1068281035.py[  95] : INFO  2-71 | lr : 0.000098 | val_loss : 1.5482 | val_roc_auc : 0.7698 | data_load_times : 44.65 | batch_run_times : 45.24
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training will continue...
[2022-12-06 22:02:40] 1068281035.py[  95] : INFO  2-72 | lr : 0.000098 | val_loss : 1.7957 | val_roc_auc : 0.7358 | data_load_times : 43.83 | batch_run_times : 44.39
Trainer was signaled to stop but required minimum epochs (75) or minimum steps (None) has not been met. Training

## Validation Inference

In [7]:
# Get model run path and define chosen fold
log_dir = "../logs/logs"
#model_run = "2022_11_08_14:57:52" # manually choose model path
model_run = hparams.log_name
model_run_path = os.path.join(log_dir, model_run)
#best_fold = 1 # manually choose model path
best_fold = val_loss_scores.index(min(val_loss_scores))

# Get best model for chosen fold
model_run_dir = os.listdir(model_run_path)
model_folds = [i for i in model_run_dir if i.startswith(f"fold={best_fold}")]
model_folds_scores = [float(i.split("val_loss=")[1].split("-")[0]) for i in model_folds]
model_name = model_folds[model_folds_scores.index(min(model_folds_scores))]
model_path = os.path.join(model_run_path, model_name)

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(model_path)["state_dict"]
)
model.eval()

# Retrieve validation indices for chosen fold
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    if fold_i == best_fold:
        break

# Select fold validation images
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Create predictions looped by batch
counter = 0
val_i_batch = []
val_idx_batch = []
scores_df = pd.DataFrame()

for i, idx in tqdm(enumerate(val_index)):
    counter += 1
    val_i_batch.append(i) # arrays don't preserve index so need ordered index values
    val_idx_batch.append(idx) # for preserved index
    
    # Run inference for val_batch_size
    if counter == hparams.val_batch_size:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        # Reset counter and batch
        counter = 0
        val_i_batch = []
        val_idx_batch = []
        
    # Run inference for remaining batch
    elif idx == val_index[-1]:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

        
# Write predictions to log
scores_df.to_csv(
    os.path.join(model_run_path, f"{model_run}_preds_fold_{best_fold}.csv"),
    index=False
)

99it [07:22,  4.47s/it]


In [8]:
scores_df

,img_path,label,ant,bedbug,bee,horsefly,mite,mosquito,none,tick
0,../data/cleaned/none/74c8654309dbd09440342475d...,none,0.083133,0.138881,0.131982,0.081957,0.095272,0.169369,0.178437,0.120969
1,../data/cleaned/none/9bac4720af91cc18252051d7f...,none,0.082288,0.136711,0.134834,0.081927,0.095663,0.169273,0.185182,0.114123
2,../data/cleaned/none/c0b5bea99fc035e3f866248c1...,none,0.082662,0.136953,0.134921,0.081989,0.095703,0.168804,0.185115,0.113853
3,../data/cleaned/none/86526ab4cf5a3497b3023b50b...,none,0.081248,0.139833,0.134338,0.081584,0.095605,0.170087,0.182911,0.114394
4,../data/cleaned/none/773cb2eaaccc890f795bd41c6...,none,0.082324,0.136545,0.133991,0.081903,0.095728,0.169920,0.184492,0.115096
...,...,...,...,...,...,...,...,...,...,...
94,../data/cleaned/ant/8b40d17bf065d297f372ad607b...,ant,0.084748,0.137350,0.132878,0.082729,0.096036,0.165352,0.183974,0.116934
95,../data/cleaned/ant/66cd2ff3237ef14490c0804123...,ant,0.083198,0.136710,0.135090,0.082343,0.095860,0.169204,0.183731,0.113864
96,../data/cleaned/ant/eba0beda4c8c60dd7dda15dec3...,ant,0.082176,0.137422,0.134570,0.081913,0.095459,0.169223,0.184847,0.114390
97,../data/cleaned/ant/dea1299d7838a747161b70c820...,ant,0.081531,0.137914,0.133155,0.081315,0.095475,0.171949,0.182595,0.116065


## Validation Analysis

In [9]:
print(f"{len(scores_df['img_path'].unique())} unique image paths.")

99 unique image paths.


In [10]:
print("Validation label counts:")
print(scores_df["label"].value_counts())

Validation label counts:
bedbug      19
tick        18
mosquito    15
ant         15
none         9
horsefly     9
mite         7
bee          7
Name: label, dtype: int64


In [11]:
print("Validation prediction counts:")
print(
    pd.melt(
        scores_df,
        id_vars=["img_path", "label"],
        value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
        var_name="pred_label",
        value_name="pred_prob"
    ).sort_values(["img_path", "pred_prob"], ascending=False) \
    .groupby(["img_path", "label"]).first()["pred_label"] \
    .value_counts()
)

Validation prediction counts:
none        98
mosquito     1
Name: pred_label, dtype: int64


In [12]:
# Probability stats by label
pd.concat(
    [
        pd.DataFrame(scores_df.iloc[:, 2:].mean(), columns=["mean"]),
        pd.DataFrame(scores_df.iloc[:, 2:].std(), columns=["std"]),
        pd.DataFrame(scores_df.iloc[:, 2:].min(), columns=["min"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.25)),
        pd.DataFrame(scores_df.iloc[:, 2:].median(), columns=["median"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.75)),
        pd.DataFrame(scores_df.iloc[:, 2:].max(), columns=["max"]),
        pd.DataFrame(scores_df.iloc[:, 2:].max() - scores_df.iloc[:, 2:].min(), columns=["range"])
    ], 
    axis=1
)

,mean,std,min,0.25,median,0.75,max,range
ant,0.082265,0.001528,0.075970,0.081522,0.082295,0.082958,0.087337,0.011367
bedbug,0.137932,0.002112,0.133489,0.136616,0.137398,0.138701,0.144628,0.011139
bee,0.134344,0.002113,0.131464,0.133146,0.133865,0.134697,0.144603,0.013139
horsefly,0.082061,0.000797,0.078521,0.081707,0.081962,0.082328,0.085161,0.006640
mite,0.095537,0.000354,0.094720,0.095283,0.095516,0.095776,0.096477,0.001757
mosquito,0.169237,0.003050,0.158445,0.168375,0.169227,0.170536,0.182665,0.024219
none,0.183353,0.002575,0.174539,0.182409,0.183913,0.184879,0.187764,0.013226
tick,0.115271,0.002626,0.103966,0.114062,0.115589,0.116928,0.120969,0.017003


In [13]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).pivot_table(
    index=["label"],
    columns=["pred_label"],
    aggfunc="mean"
)

pred_prob                                                    \
pred_label       ant    bedbug       bee  horsefly      mite  mosquito   
label                                                                    
ant         0.082235  0.138594  0.134759  0.082191  0.095583  0.168426   
bedbug      0.082261  0.137662  0.134332  0.081991  0.095520  0.169335   
bee         0.082197  0.138357  0.135027  0.082107  0.095498  0.169536   
horsefly    0.081468  0.137982  0.134171  0.081678  0.095625  0.170763   
mite        0.081489  0.138339  0.133969  0.081791  0.095499  0.170642   
mosquito    0.082250  0.137922  0.134078  0.082058  0.095364  0.169307   
none        0.082256  0.137637  0.134124  0.081868  0.095560  0.169779   
tick        0.083037  0.137472  0.134307  0.082405  0.095636  0.168057   

                                
pred_label      none      tick  
label                           
ant         0.183358  0.114855  
bedbug      0.183757  0.115142  
bee         0.182913  0.114366  
horsefly    0.183097  0.115215  
mite        0.182797  0.115474  
mosquito    0.183243  0.115778  
none        0.183478  0.115299  
tick        0.183469  0.115616

In [14]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).sort_values(["img_path", "pred_prob"], ascending=False).groupby(["img_path", "label"]).first()

,,pred_label,pred_prob
img_path,label,,
../data/cleaned/ant/09201674df1942ec6433a487f195cca68f23310b.jpg,ant,none,0.179889
../data/cleaned/ant/0cf3675aae8601ba680bd9585f8023f66d56a771.jpg,ant,none,0.179859
../data/cleaned/ant/100eac5fb92b879adc0bda8e26b65f9b9fed5099.jpg,ant,none,0.184605
../data/cleaned/ant/4adf2283fed4f3060a8be1e516d1005e21dd1c42.jpg,ant,none,0.183353
../data/cleaned/ant/62830f28c6bfb81efd3d9bf0a3db6f667dec2dad.jpg,ant,none,0.182438
...,...,...,...
../data/cleaned/tick/9e71cd0ae23e77c93c596c65f2e44b6b14dbc62b.jpg,tick,none,0.183913
../data/cleaned/tick/a5db00f3302356054e3f1ba1fb557511faf905a4.jpg,tick,none,0.187764
../data/cleaned/tick/df5025f8bc1363330c965e950fd4aa950efee715.jpg,tick,none,0.184202
